In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

In [30]:
labevents     = pd.read_parquet(hosp+'labevents.parquet')
d_labitems    = pd.read_parquet(hosp+'d_labitems.parquet')

patients      = pd.read_parquet(hosp+'patients.parquet')
admissions    = pd.read_parquet(hosp+'admissions.parquet')

diagnoses_icd = pd.read_parquet(hosp+'diagnoses_icd.parquet')
microbiology  = pd.read_parquet(hosp+'microbiologyevents.parquet')
prescriptions = pd.read_parquet(hosp+'prescriptions.parquet')

In [3]:
chartevents     = pd.read_parquet(icu+'chartevents.parquet')
d_items         = pd.read_parquet(icu+'d_items.parquet')
inputevents     = pd.read_parquet(icu+'inputevents.parquet')
procedureevents = pd.read_parquet(icu+'procedureevents.parquet')
icustays        = pd.read_parquet(icu+'icustays.parquet')

In [20]:
chartevents['charttime'] = pd.to_datetime(chartevents['charttime'])

In [10]:
d_antibiotics = d_items[(d_items['linksto']=='inputevents')&(d_items['category']=='Antibiotics')].itemid.unique()

hadm_id_AB = inputevents[inputevents['itemid'].isin(d_antibiotics)].hadm_id.unique()
stay_id_AB = inputevents[inputevents['itemid'].isin(d_antibiotics)].stay_id.unique()

print("hadm_id_AB: ",len(hadm_id_AB))
print("stay_id_AB: ",len(stay_id_AB))

hadm_id_AB:  41063
stay_id_AB:  44636


In [28]:
chartevents_SCr = chartevents[chartevents['itemid'].isin([220615])]
chartevents_SCr = chartevents_SCr[(chartevents_SCr['valuenum'] <= 40) & (chartevents_SCr['valuenum'] >= 0.1)]

chartevents_SCr.sort_values(by=['subject_id','charttime'],ascending=True,inplace=True)
chartevents_SCr.reset_index(inplace=True, drop=True)

chartevents_SCr = chartevents_SCr[['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

In [29]:
chartevents_SCr

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum
0,10000032,29079034,39553978,2180-07-23 21:45:00,220615,0.5
1,10001217,24597018,37067082,2157-11-21 03:16:00,220615,0.4
2,10001217,27703517,34592300,2157-12-20 01:45:00,220615,0.5
3,10001725,25563031,31205490,2110-04-11 18:02:00,220615,0.8
4,10001725,25563031,31205490,2110-04-12 02:59:00,220615,0.8
...,...,...,...,...,...,...
425251,19999840,21033226,38978960,2164-09-17 02:49:00,220615,0.8
425252,19999840,21033226,38978960,2164-09-17 07:55:00,220615,0.8
425253,19999840,21033226,38978960,2164-09-17 13:13:00,220615,0.7
425254,19999987,23865745,36195440,2145-11-03 01:35:00,220615,1.3
